In [1]:
%matplotlib inline
import backtrader as bt
import pandas as pd


In [2]:
daily = pd.read_csv('AAPL.csv', index_col=0, parse_dates=True)
data = bt.feeds.PandasData(dataname=daily.sort_index())

In [3]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [4]:
cerebro = bt.Cerebro()

cerebro.broker.setcash(100000.0)

# data = bt.feeds.GenericCSVData(
#   dataname='AAPL.csv',
#   dtformat=("%m/%d/%Y"),
#   datetime=0,
#   open=3,
#   high=4,
#   low=5,
#   close=1,
#   volume=2,
#   openinterest=-1
#   )



cerebro.adddata(data)
cerebro.addstrategy(TestStrategy)


print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()


print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00
2014-07-07, Close, 39.34
2014-07-07, BUY CREATE, 39.34
2014-07-08, BUY EXECUTED, 39.32
2014-07-08, Close, 39.28
2014-07-09, Close, 39.73
2014-07-10, Close, 39.42
2014-07-11, Close, 39.30
2014-07-14, Close, 39.28
2014-07-15, Close, 39.45
2014-07-15, SELL CREATE, 39.45
2014-07-16, SELL EXECUTED, 39.53
2014-07-16, Close, 39.37
2014-07-17, Close, 38.62
2014-07-17, BUY CREATE, 38.62
2014-07-18, BUY EXECUTED, 38.85
2014-07-18, Close, 38.97
2014-07-21, Close, 38.80
2014-07-22, Close, 39.37
2014-07-23, Close, 39.57
2014-07-24, Close, 40.23
2014-07-25, Close, 39.37
2014-07-25, SELL CREATE, 39.37
2014-07-28, SELL EXECUTED, 39.40
2014-07-28, Close, 39.18
2014-07-28, BUY CREATE, 39.18
2014-07-29, BUY EXECUTED, 39.24
2014-07-29, Close, 39.33
2014-07-30, Close, 39.45
2014-07-31, Close, 38.84
2014-08-01, Close, 38.49
2014-08-04, Close, 38.77
2014-08-05, Close, 38.40
2014-08-05, SELL CREATE, 38.40
2014-08-06, SELL EXECUTED, 38.25
2014-08-06, Close, 38.56
2014-08-07,

In [5]:
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]